In [1]:
#Asegurar que el código sea reproducible
import random
import numpy as np
import torch

SEED = 3633

#Semilla para Python
random.seed(SEED)

#Semilla para NumPy
np.random.seed(SEED)

# Semilla para PyTorch
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
#Cargamos los modelos y datos
import pickle
import pandas as pd

with open("modelos/kmeans_games.pkl", "rb") as f:
    kmeans_games = pickle.load(f)
with open("modelos/kmeans_players.pkl", "rb") as f:
    kmeans_players = pickle.load(f)

player_embeddings = pd.read_csv("csvs/player_embeddings.csv")
games = pd.read_csv("csvs/games_metadata_gnn.csv")
games_embeddings = pd.read_csv("csvs/embeddings_gnn.csv")

c:\Users\B\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator KMeans from version 1.0.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\B\AppData\Local\Temp\ipykernel_1548\1112781581.py:11: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  games = pd.read_csv("csvs/games_metadata_gnn.csv")


In [3]:
import os
import zipfile
import chess.pgn
from io import TextIOWrapper
from torch_geometric.data import Data




# =====================
# FUNCIÓN DE FEATURES
# =====================

def board_to_feature(board):
    """Convierte el estado del tablero a un vector de 773 features"""
    piece_map = board.piece_map()
    planes = np.zeros((12, 64), dtype=np.float32)
    piece_to_index = {
        'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
        'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
    }
    for square, piece in piece_map.items():
        planes[piece_to_index[piece.symbol()]][square] = 1.0
    flat_pieces = planes.reshape(-1)  # 768

    extras = [
        float(board.turn),
        float(board.has_kingside_castling_rights(chess.WHITE)),
        float(board.has_queenside_castling_rights(chess.WHITE)),
        float(board.has_kingside_castling_rights(chess.BLACK)),
        float(board.has_queenside_castling_rights(chess.BLACK)),
    ]
    return np.concatenate([flat_pieces, extras])  # 773

# =====================
# FUNCIÓN PARA GRAFO DE PARTIDA
# =====================

def pgn_to_graph_one_player(game, color):
    board = game.board()
    x = []
    edge_index = [[], []]
    node_idx = 0
    move_idx = 0

    for move in game.mainline_moves():
        board.push(move)
        if (color == "white" and move_idx % 2 == 0) or (color == "black" and move_idx % 2 == 1):
            x.append(board_to_feature(board.copy()))
            if node_idx > 0:
                edge_index[0].append(node_idx - 1)
                edge_index[1].append(node_idx)
            node_idx += 1
        move_idx += 1

    if len(x) < 2:
        return None

    x = torch.tensor(np.stack(x), dtype=torch.float)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    return Data(x=x, edge_index=edge_index)

def process_single_pgn(pgn_path, target_player):
    data_graphs = []
    game_id = 0

    with open(pgn_path, encoding='utf-8', errors='ignore') as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break

            white = game.headers.get("White", "").lower()
            black = game.headers.get("Black", "").lower()
            target = target_player.lower()

            if target == white:
                color = "white"
            elif target == black:
                color = "black"
            else:
                print(f"El jugador {target_player} no está en esta partida.")
                continue

            graph = pgn_to_graph_one_player(game, color)
            if graph:
                graph.player = target_player
                graph.game_id = game_id
                data_graphs.append(graph)
                game_id += 1  
    print(f"Utilizando {len(data_graphs)} partidas de {target_player}")
    return data_graphs

c:\Users\B\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader
import pandas as pd
from tqdm import tqdm

# ======================
# Hiperparámetros
# ======================
INPUT_DIM = 773
HIDDEN_DIM = 256
EMBED_DIM = 128
BATCH_SIZE = 32

# ======================
# Modelo GCN Encoder
# ======================
class GNNEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return global_mean_pool(x, batch)




In [5]:
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity


def make_recommendations(player_name, pgn_path, n_players, n_games):
    print("Leyendo partidas...")
    #Crear grafos
    graphs_list = process_single_pgn(pgn_path, player_name)
    # Cargar grafos
    loader = DataLoader(graphs_list, batch_size=BATCH_SIZE, shuffle=False)

    model = GNNEncoder(INPUT_DIM, HIDDEN_DIM, EMBED_DIM)
    model.eval()

    # ======================
    # Embedding loop
    # ======================
    embeddings = []
    players = []
    game_ids = []

    with torch.no_grad():
        for batch in tqdm(loader, desc="Generando embeddings"):
            z = model(batch.x, batch.edge_index, batch.batch)
            embeddings.append(z)
            graphs_in_batch = batch.to_data_list()
            players += [g.player for g in graphs_in_batch]
            game_ids += [g.game_id for g in graphs_in_batch]

    embeddings = torch.cat(embeddings).cpu().numpy()


    data = pd.DataFrame(embeddings, columns=[f"dim_{i}" for i in range(EMBED_DIM)])
    embedding_columns = [f"dim_{i}" for i in range(128)]
    data['embedding'] = data[embedding_columns].values.tolist()
    X = np.stack(data['embedding'].values)

    labels = kmeans_games.predict(X)

    
    #Crear vector para el usuario

    labels = kmeans_games.predict(X)
    counts = Counter(labels)
    cluster_vector = np.zeros(kmeans_games.n_clusters)

    for cluster_id, count in counts.items():
        cluster_vector[cluster_id] = count

    cluster_vector = cluster_vector / cluster_vector.sum()

    #Recomendacion de Estilo (Cluster)
    estilo = kmeans_players.predict(cluster_vector.reshape(1, -1))[0]
    print("\nTu estilo es:")
    if estilo == 0:
        print("🟦 Cluster 0 – “Vanguardia del Ajedrez Moderno”")
        print("Descripción: Jugadores que transicionaron entre el ajedrez clásico y el pensamiento moderno. Mezclan elegancia posicional con creatividad táctica.")
        print("Estilo: Creatividad estructurada, técnica refinada, inicio del pensamiento hipermoderno.")
        print("Ejemplos: Alekhine, Bogoljubow, Capablanca, Nimzowitsch, Rubinstein")

    elif estilo == 1:
        print("🟩 Cluster 1 – “Escuela Clásica y Fundacional”")
        print("Descripción: Padres fundadores del ajedrez posicional. Énfasis en principios clásicos como el control central y el desarrollo armónico.")
        print("Estilo: Clásico, científico, riguroso.")
        print("Ejemplos: Lasker, Steinitz, Tarrasch, Staunton, Chigorin")

    elif estilo == 2:
        print("🟨 Cluster 2 – “Maestros Contemporáneos del Siglo XXI”")
        print("Descripción: Jugadores modernos con un estilo universal. Alta preparación teórica, precisión y flexibilidad.")
        print("Estilo: Universal, multifacético, técnico y adaptativo.")
        print("Ejemplos: Carlsen, Anand, Kramnik, Caruana, Topalov")

    elif estilo == 3:
        print("🟠 Cluster 3 – “Escuela Posicional Soviética”")
        print("Descripción: Jugadores de la tradición soviética. Muy técnicos, con enfoque estratégico y profundo conocimiento del medio juego.")
        print("Estilo: Posicional, estructural, metódico.")
        print("Ejemplos: Botvinnik, Karpov, Petrosian, Smyslov, Gelfand")

    elif estilo == 4:
        print("🔴 Cluster 4 – “Genios Creativos y Agresivos”")
        print("Descripción: Grandes calculadores y atacantes. Estilo impredecible, dinámico y espectacular.")
        print("Estilo: Táctico, agresivo, intuitivo.")
        print("Ejemplos: Fischer, Kasparov, Tal, Bronstein, Spassky")

    else:
        print("Error")
        return


    

    #Hacer Recomendaciones de jugadores
    print("\nJugadores similares recomendados:\n")
    player_names = player_embeddings["player"]
    X_embeddings = player_embeddings.drop(columns=["player"]).values
    sims = cosine_similarity([cluster_vector], X_embeddings)[0]
    # Crear dataframe ordenado por similitud
    similarities_df = pd.DataFrame({    
        "player": player_names,
        "similarity": sims
    }).sort_values(by="similarity", ascending=False)

    recommended_players = list(similarities_df.head(n_players)["player"])
    similarities = list(similarities_df.head(n_players)["similarity"])


    #Para hacer recomendaciones de juegos
    vector_promedio = np.mean(X, axis=0).reshape(1, -1)

    for n in range(n_players):
        recommended_player = recommended_players[n]
        print(f"({n + 1}) {recommended_player} con {round(similarities[n], 2)} de similitud\n")
        print("Partidas recomendadas:\n")
        juegos = games_embeddings[games_embeddings["player"] == recommended_player].copy()


        dim_cols = [col for col in juegos.columns if col.startswith("dim_")]
        embeddings = juegos[dim_cols].values  # Convertir a matriz NumPy

    
        # Calcular similitudes de coseno entre vector promedio y juegos
        similitudes = cosine_similarity(vector_promedio, embeddings)[0]  # Solo una fila vs todas

        #Añadir similitudes al DataFrame
        juegos['cosine_similarity'] = similitudes

        #Obtener los |n_games| vectores más similares
        top = juegos.sort_values(by='cosine_similarity', ascending=False).head(n_games)
        
        ids_juegos = list(top["game_id"])
        resultados = games[games["game_id"].isin(ids_juegos)][["game_id", "pgn"]]

        pgns = list(resultados["pgn"])
        for p in pgns:
            print(p)
            print("\n")
        print("\n")

        
        
        
        
    


    
    
    
    
      


In [6]:
make_recommendations("felipepollo" , "lichess/lichess_felipepollo_2025-06-29.pgn" , 1, 1)

Leyendo partidas...
Utilizando 3386 partidas de felipepollo


Generando embeddings: 100%|██████████| 106/106 [00:01<00:00, 95.61it/s] 



Tu estilo es:
🟠 Cluster 3 – “Escuela Posicional Soviética”
Descripción: Jugadores de la tradición soviética. Muy técnicos, con enfoque estratégico y profundo conocimiento del medio juego.
Estilo: Posicional, estructural, metódico.
Ejemplos: Botvinnik, Karpov, Petrosian, Smyslov, Gelfand

Jugadores similares recomendados:

(1) Korchnoi con 0.81 de similitud

Partidas recomendadas:

[Event "4th Pivdenny Bank Chess Cup"]
[Site "Odessa UKR"]
[Date "2008.05.31"]
[Round "8"]
[White "Beim,V"]
[Black "Korchnoi,V"]
[Result "1/2-1/2"]
[WhiteElo "2523"]
[BlackElo "2598"]
[ECO "D42"]

1. Nf3 e6 2. d4 d5 3. c4 Nf6 4. Nc3 c5 5. e3 Nc6 6. cxd5 Nxd5 7. Bd3 cxd4 8. exd4 Bb4 9. Bd2 Be7 10. a3 Bf6 11. Be3 Nxc3 12. bxc3 e5 13. dxe5 Nxe5 14. Nxe5 Bxe5 15. O-O Qc7 16. Bb5+ Bd7 17. Bxd7+ Qxd7 18. Qh5 Qc7 19. Bd4 Bxd4 20. Rfe1+ Kf8 21. cxd4 Rd8 22. Rac1 Qd6 23. Qa5 g6 24. Qxa7 Rd7 25. h3 Kg7 26. Rcd1 Rc8 27. Qa5 Rc4 28. Qe5+ Qxe5 29. dxe5 Rxd1 30. Rxd1 Rc3 31. Rb1 Rxa3 32. Rxb7 Kg8 33. Rb8+ Kg7 34. Rb7 1/2-1